<a href="https://colab.research.google.com/github/ChiefGokhlayeh/aibv/blob/master/lab4/lab4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Labor 4 Deep Learning für die Solarzelleninspektion mit CNN

Im Kurs **(IE/DHL) Angewandte industrielle Bildverarbeitung S22** der **Hochschule für Angewandte Wissenschaften Hamburg**. Durchgeführt von:

* **Andreas Baulig**
* **Fabian Mahler**

Laborbetreuung:

* **Prof. Dr.-Ing. Dipl.-Kfm. J. Dahlkemper**

## Ziel

In diesem Versuch soll ein auf der Bilddatenbank ImageNet pre-trained CNN mit einem von Grund auf trainiertem CNN im Hinblick auf die Leistungsfähigkeit der Klassifizierung von Solarzellen auf Basis derer Elektrolumineszenz-Bilder verglichen werden.


## Vorbereitung

Zunächst wird geprüft, ob die für das Labor notwendigen Python Pakete installiert sind. Darunter fällt auch ein Test, ob TensorFlow mit GPU-Beschleunigung arbeiten kann.


In [ ]:
import tensorflow as tf

# Check if GPU is present
if tf.test.gpu_device_name():
    print(f"Default GPU Device: {tf.test.gpu_device_name()}")
else:
    print("Please install GPU version of TF")

**NUR IN GOOGLE COLAB!**


In [ ]:
try:
    import google.colab

    IN_COLAB = True
    print("Running in Google Colab.")
except ImportError:
    IN_COLAB = False
    print("Not running in Google Colab.")

In [ ]:
if IN_COLAB:
    from google.colab import drive

    drive.mount("/content/drive")

In [ ]:
import requests
import zipfile
import io
import os
from tqdm import tqdm


def extract_dataset(fileobj, data_dir):
    with zipfile.ZipFile(fileobj) as z:
        for member in tqdm(
            z.infolist(), desc=f"Extracting to {os.path.normpath(data_dir)}"
        ):
            z.extract(member, data_dir)


if IN_COLAB:
    # In Google Colab we have to first download the zip file from the
    # repository.
    data_dir = "/content/data"
    dataset_source_url = "https://media.githubusercontent.com/media/ChiefGokhlayeh/aibv/master/data/multilabel.zip"
    with requests.get(dataset_source_url) as req:
        req.raise_for_status()

        extract_dataset(io.BytesIO(req.content), data_dir)

else:
    # Assume we are executing within the repository, so the zip file should be
    # available locally.
    data_dir = "../data"
    dataset_source_path = "../data/multilabel.zip"

    with open(dataset_source_path, "r") as f:
        extract_dataset(f, data_dir)

multilabel_dir = os.path.join(data_dir, "multilabel")

## Bildtransformationen mit `ImageDataGenerator`

Überlegen und schriftlich festhalten, welche Arten der im `ImageDataGenerator` verfügbaren Bildtransformationen (siehe https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator) für die Data Augmentation der Solarzellenbilder geeignet ist, (detailliertes Tutorial hierzu https://towardsdatascience.com/image-augmentationfor-deep-learning-using-keras-and-histogram-equalization-9329f6ae5085).


In [ ]:
import itertools
import pandas as pd
import matplotlib.pyplot as plt

from keras.preprocessing.image import ImageDataGenerator

labels_df = pd.read_csv(os.path.join(multilabel_dir, "labels.csv"))

image_gen = ImageDataGenerator(
    rotation_range=0,
    width_shift_range=0.0085,
    height_shift_range=0.009,
    brightness_range=(0.3, 1.1),
    shear_range=0.1,
    fill_mode="constant",
    cval=0,
    horizontal_flip=True,
    rescale=1.0 / 255.0,
    validation_split=0.7,
)

imggen_sample_size = 5
sample_batch_size = 3

label_names = ["crack", "darkarea", "finger"]

gen_iter = image_gen.flow_from_dataframe(
    dataframe=labels_df,
    directory=multilabel_dir,
    x_col="File",
    y_col=label_names,
    batch_size=sample_batch_size,
    color_mode="rgb",
    class_mode="multi_output",
)

fig, axs = plt.subplots(sample_batch_size, imggen_sample_size, figsize=(15, 9))

for i, (batch, labels) in enumerate(itertools.islice(gen_iter, imggen_sample_size)):
    for j, img in enumerate(batch):
        ax = axs[j, i]
        ax.imshow(batch[j, :, :, :], cmap="gray")
        labels_s = pd.Series(labels[j], index=label_names)
        ax.set_title(", ".join(labels_s.index[labels_s == 1].to_list()) or "ok")

fig.tight_layout()

## **Meta**: Google Colab Code Formatierung und Aufräumen

Erlaubt es die Codezellen dieses Notebooks zu formatieren und etwaige Zellinhalte zu löschen. Besonderes letzteres sollte vor jedem `git commit` durchgeführt werden, um die `.ipynb` Datei kleinzuhalten.

Nach dem Ausführen der Tools **nicht speichern**, stattdessen Seite mit `F5` neu laden. **Nur** für Verwendung in Google Colab gedacht. 

Inspiriert von https://stackoverflow.com/a/71001241.


In [ ]:
if IN_COLAB:
    %pip install black[jupyter] nbstripout
    !black '/content/drive/MyDrive/Colab Notebooks/lab4.ipynb'
    !nbstripout '/content/drive/MyDrive/Colab Notebooks/lab4.ipynb'